In [1]:
import minitorch

## Go Through Minitorch

This is a high level abstraction of some basic components in MiniTorch.

<img src="../imgs/high_level_abstraction.png"></img>

`class Parameter` is a very important component defined in `class Module`. What the models are learing is actually these `Parameter`(s). Optimizers also look for these parameters and update them following the weight updating rules. See `minitorch/module.py`.

In [7]:
from typing import Any, Dict, Optional, Sequence, Tuple

class Parameter:
    """
    A Parameter is a special container stored in a `Module`.

    It is designed to hold a `Variable`, but we allow it to hold
    any value for testing.
    """

    def __init__(self, x: Any, name: Optional[str] = None) -> None:
        self.value = x
        self.name = name
        if hasattr(x, "requires_grad_"):
            self.value.requires_grad_(True)
            if self.name:
                self.value.name = self.name

    def update(self, x: Any) -> None:
        "Update the parameter value."

In [8]:
class Module:
    """
    Modules form a tree that store parameters and other
    submodules. They make up the basis of neural network stacks.

    Attributes:
        _modules : Storage of the child modules
        _parameters : Storage of the module's parameters
        training : Whether the module is in training mode or evaluation mode

    """

    _modules: Dict[str, 'Module']
    _parameters: Dict[str, Parameter]
    training: bool

    def parameters(self) -> Sequence[Parameter]:
        "Enumerate over all the parameters of this module and its descendents."
        return [j for _, j in self.named_parameters()]

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        return self.forward(*args, **kwargs)

    def train(self) -> None:
        "Set the mode of this module and all descendent modules to `train`."
        for m in self.modules():
            m.train()
        self.training = True

    def eval(self) -> None:
        "Set the mode of this module and all descendent modules to `eval`."
        for m in self.modules():
            m.eval()
        self.training = False

We define a `RParam` function here to initialize the `Parameter` in `project/run_sentiment.py`. We can define more functions for initialization i.e. [xavier](https://paperswithcode.com/method/xavier-initialization), [kaiming](https://paperswithcode.com/method/he-initialization) and structure them better inside the minitorch.

In [9]:
# Initialization
def RParam(*shape):
    r = 0.1 * (minitorch.rand(shape, backend=BACKEND) - 0.5)
    return minitorch.Parameter(r)

See `minitorch/optim.py`.

In [10]:
class Optimizer:
    def __init__(self, parameters: Sequence[Parameter]):
        self.parameters = parameters


class SGD(Optimizer):
    def __init__(self, parameters: Sequence[Parameter], lr: float = 1.0):
        super().__init__(parameters)
        self.lr = lr

    def step(self) -> None:
        for p in self.parameters:
            if p.value is None:
                continue
            elif hasattr(p.value, "grad"):
                if p.value.grad is not None:
                    p.update(p.value - self.lr * p.value.grad)

`class Function` is an abstraction for operators with `forward` and `backward` functions. It represents the kind of computation we perform at each node. See `minitorch/tensor_functions.py`.

In [12]:
from minitorch.autodiff import Context
from minitorch.tensor import Tensor

class Function:
    @classmethod
    def _backward(cls, ctx: Context, grad_out: Tensor) -> Tuple[Tensor, ...]:
        return wrap_tuple(cls.backward(ctx, grad_out))  # type: ignore

    @classmethod
    def _forward(cls, ctx: Context, *inps: Tensor) -> Tensor:
        return cls.forward(ctx, *inps)  # type: ignore

    @classmethod
    def apply(cls, *vals: Tensor) -> Tensor:
        raw_vals = []
        need_grad = False
        for v in vals:
            if v.requires_grad():
                need_grad = True
            raw_vals.append(v.detach())

        # Create the context.
        ctx = Context(not need_grad)

        # Call forward with the variables.
        c = cls._forward(ctx, *raw_vals)

        # Create a new variable from the result with a new history.
        back = None
        if need_grad:
            back = minitorch.History(cls, ctx, vals)
        return minitorch.Tensor(c._tensor, back, backend=c.backend)

In [13]:
class Add(Function):
    @staticmethod
    def forward(ctx: Context, t1: Tensor, t2: Tensor) -> Tensor:
        return t1.f.add_zip(t1, t2)

    @staticmethod
    def backward(ctx: Context, grad_output: Tensor) -> Tuple[Tensor, Tensor]:
        return grad_output, grad_output

`class Tensor` is the basic data structure that handles multidimensioanl arrays. See `minitorch/tensor.py`.

In [14]:
from minitorch.tensor_ops import TensorBackend
from minitorch.tensor import History
from minitorch.tensor_data import TensorData

class Tensor:
    """
    Tensor is a generalization of Scalar in that it is a Variable that
    handles multidimensional arrays.
    """

    backend: TensorBackend
    history: Optional[History]
    grad: Optional[Tensor]
    _tensor: TensorData
    unique_id: int
    name: str

## Data Storage and Operators

This diagram shows several important components of `class Tensor`, including data storage `class TensorData`, backend `class TensorBackend`, etc. Backend relies on `class TensorOps` to execute the actual computation for different operators. Basic functions are implemented and abstracted by `class Function`.

<img src="../imgs/data_storage_operators.png"></img>

In [15]:
import numpy as np
import numpy.typing as npt
from typing import Sequence
from typing_extensions import TypeAlias

Storage: TypeAlias = npt.NDArray[np.float64]
OutIndex: TypeAlias = npt.NDArray[np.int32]
Index: TypeAlias = npt.NDArray[np.int32]
Shape: TypeAlias = npt.NDArray[np.int32]
Strides: TypeAlias = npt.NDArray[np.int32]

UserIndex: TypeAlias = Sequence[int]
UserShape: TypeAlias = Sequence[int]
UserStrides: TypeAlias = Sequence[int]

In [16]:
class TensorData:
    _storage: Storage
    _strides: Strides
    _shape: Shape
    strides: UserStrides
    shape: UserShape
    dims: int

### Indexing

<img src="../imgs/strides.png"></img>

In [2]:
x = minitorch.tensor([1, 2, 3, 4, 5, 6])

In [3]:
x.to_numpy()

array([1., 2., 3., 4., 5., 6.], dtype=float32)

In [4]:
x._tensor.shape, x._tensor.strides

((6,), (1,))

In [5]:
y = minitorch.Tensor.make(
    storage=x._tensor._storage,
    shape=(2, 3),
    strides=(3, 1),
    backend=x.backend)
y


[
	[1.000000 2.000000 3.000000]
	[4.000000 5.000000 6.000000]]

In [21]:
y._tensor._storage

array([1., 2., 3., 4., 5., 6.])

In [22]:
y._tensor.shape, y._tensor.strides

((2, 3), (3, 1))

In [23]:
y._tensor._storage is x._tensor._storage

True

In [25]:
z = x.view(3, 2, 1)
z


[
	[
		[1.000000]
		[2.000000]]
	[
		[3.000000]
		[4.000000]]
	[
		[5.000000]
		[6.000000]]]

In [26]:
z._tensor.shape, z._tensor.strides

((3, 2, 1), (2, 1, 1))

In [27]:
z._tensor._storage is x._tensor._storage

True

In [28]:
z_index = [1, 1, 0]
pos = minitorch.index_to_position(z_index, z._tensor._strides)
z[tuple(z_index)] == z._tensor._storage[pos], pos, z[tuple(z_index)]

(True, 3, 4.0)

In [29]:
out_index = [0, 0, 0]
minitorch.to_index(3, z.shape, out_index)
out_index

[1, 1, 0]

In [30]:
shape = (2, 3)
minitorch.strides_from_shape(shape)

(3, 1)

In [7]:
shape = (3, 2, 1)
minitorch.strides_from_shape(shape)

(2, 1, 1)

In [32]:
p = z.permute(2, 1, 0)
p


[
	[
		[1.000000 3.000000 5.000000]
		[2.000000 4.000000 6.000000]]]

In [47]:
p._tensor.shape, p._tensor.strides

((1, 2, 3), (1, 1, 2))

In [48]:
p._tensor._storage is x._tensor._storage

True

In [49]:
p_index = [0, 0, 0]
minitorch.to_index(-1, p.shape, p_index)
p_index

[0, 1, 2]

In [50]:
minitorch.index_to_position(p_index, p._tensor._strides)

5

### Backend & Operators

See `minitorch/tensor_ops.py`

In [34]:
from typing import Callable, Generic, Iterable, Tuple, TypeVar
from minitorch.tensor_ops import MapProto

class TensorOps:
    @staticmethod
    def map(fn: Callable[[float], float]) -> MapProto:
        pass

    @staticmethod
    def zip(fn: Callable[[float, float], float]) -> Callable[[Tensor, Tensor], Tensor]:
        pass

    @staticmethod
    def reduce(
        fn: Callable[[float, float], float], start: float = 0.0
    ) -> Callable[[Tensor, int], Tensor]:
        pass

    @staticmethod
    def matrix_multiply(a: Tensor, b: Tensor) -> Tensor:
        raise NotImplementedError("Not implemented in this assignment")

    @staticmethod
    def cmap(fn: Callable[[float], float]) -> Callable[[Tensor, Tensor], Tensor]:
        pass
    cuda = False

In [35]:
from minitorch.tensor import operators
from typing import Type

class TensorBackend:
    def __init__(self, ops: Type[TensorOps]):
        # Maps
        self.neg_map = ops.map(operators.neg)
        self.sigmoid_map = ops.map(operators.sigmoid)
        self.relu_map = ops.map(operators.relu)
        self.log_map = ops.map(operators.log)
        self.exp_map = ops.map(operators.exp)
        self.id_map = ops.map(operators.id)
        self.id_cmap = ops.cmap(operators.id)
        self.inv_map = ops.map(operators.inv)

        # Zips
        self.add_zip = ops.zip(operators.add)
        self.mul_zip = ops.zip(operators.mul)
        self.lt_zip = ops.zip(operators.lt)
        self.eq_zip = ops.zip(operators.eq)
        self.is_close_zip = ops.zip(operators.is_close)
        self.relu_back_zip = ops.zip(operators.relu_back)
        self.log_back_zip = ops.zip(operators.log_back)
        self.inv_back_zip = ops.zip(operators.inv_back)

        # Reduce
        self.add_reduce = ops.reduce(operators.add, 0.0)
        self.mul_reduce = ops.reduce(operators.mul, 1.0)

        # Matrix Multiply
        self.matrix_multiply = ops.matrix_multiply


In [36]:
class CudaKernelOps(TensorOps):
    @staticmethod
    def map(fn: Callable[[float], float]) -> MapProto:
        ### Your Implementation ###
        pass

    @staticmethod
    def zip(fn: Callable[[float, float], float]) -> Callable[[Tensor, Tensor], Tensor]:
        ### Your Implementation ###
        pass

    @staticmethod
    def reduce(
        fn: Callable[[float, float], float], start: float = 0.0
    ) -> Callable[[Tensor, int], Tensor]:
        ### Your Implementation ###
        pass

    @staticmethod
    def matrix_multiply(a: Tensor, b: Tensor) -> Tensor:
        ### Your Implementation ###
        pass

In [37]:
# In project/run_sentiment.py
backend_name = "CudaKernelOps"

if backend_name == "CudaKernelOps":
    #from minitorch.cuda_kernel_ops import CudaKernelOps
    BACKEND = minitorch.TensorBackend(CudaKernelOps)

### Connect to CUDA Kernels

In [81]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp311-cp311-linux_x86_64.whl size=660362 sha256=1328ab5c10437cabc800b3c34d93cc91ae72f3df2d1ff69d9b5001a96c2e537f
  Stored in directory: /root/.cache/pip/wheels/76/66/50/c65e6116d7e0e16abe0f7c19b50327f76724ccfefbdc61a1b9
Successfully built pycuda


In [82]:
import ctypes
import pycuda.gpuarray as gpuarray
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [83]:
try:
    lib = ctypes.CDLL("minitorch/cuda_kernels/combine.so")
except:
    print("cuda kernels not implemented: combine.so not found")

cuda kernels not implemented: combine.so not found


Inside cuda_kernel_ops.py, we call cuda kernels

In [84]:
def map(fn: Callable[[float], float]) -> MapProto:
    "See `tensor_ops.py`"
    fn_id = fn_map[fn]

    def ret(a: Tensor, out: Optional[Tensor] = None) -> Tensor:
        if out is None:
            out = a.zeros(a.shape)

        # Define the argument type for the tensorMap function
        lib.tensorMap.argtypes = [
            ctypes.POINTER(ctypes.c_double),  # out
            ctypes.POINTER(ctypes.c_int),    # out_shape
            ctypes.POINTER(ctypes.c_int),    # out_strides
            ctypes.c_int,                    # out_size
            ctypes.POINTER(ctypes.c_double),  # in_storage
            ctypes.POINTER(ctypes.c_int),    # in_shape
            ctypes.POINTER(ctypes.c_int),    # in_strides
            ctypes.c_int,                    # shape_len
            ctypes.c_int,                    # fn_id
        ]

        # Define the return type for the tensorMap function
        lib.tensorMap.restype = None

        # Convert the numpy arrays to gpuarrays that can be loaded to the gpu
        out_array_gpu = gpuarray.to_gpu(out._tensor._storage)
        out_shape_gpu = gpuarray.to_gpu(out._tensor._shape.astype(np.int32))
        out_strides_gpu = gpuarray.to_gpu(out._tensor._strides.astype(np.int32))
        in_array_gpu = gpuarray.to_gpu(a._tensor._storage)
        in_shape_gpu = gpuarray.to_gpu(a._tensor._shape.astype(np.int32))
        in_strides_gpu = gpuarray.to_gpu(a._tensor._strides.astype(np.int32))


        # Call the function
        lib.tensorMap(
            ctypes.cast(out_array_gpu.ptr, ctypes.POINTER(ctypes.c_double)),
            ctypes.cast(out_shape_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
            ctypes.cast(out_strides_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
            ctypes.c_int(out.size),
            ctypes.cast(in_array_gpu.ptr, ctypes.POINTER(ctypes.c_double)),
            ctypes.cast(in_shape_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
            ctypes.cast(in_strides_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
            ctypes.c_int(len(a.shape)),
            ctypes.c_int(fn_id)
        )

        # Copy the gpuarray back to the cpu
        out._tensor._storage = out_array_gpu.get()

        # Free the gpuarrays
        out_array_gpu.gpudata.free()
        out_shape_gpu.gpudata.free()
        out_strides_gpu.gpudata.free()
        in_array_gpu.gpudata.free()
        in_shape_gpu.gpudata.free()
        in_strides_gpu.gpudata.free()
        return out

    return ret

```c++
    void tensorMap(
        scalar_t* out,
        int* out_shape,
        int* out_strides,
        int out_size,
        scalar_t* in_storage,
        int* in_shape,
        int* in_strides,
        int shape_size,
        int fn_id
    ) {
        int threadsPerBlock = BASE_THREAD_NUM;
        int blocksPerGrid = (out_size + threadsPerBlock - 1) / threadsPerBlock;
        mapKernel<<<blocksPerGrid, threadsPerBlock>>>(out, out_shape, out_strides, out_size, in_storage, in_shape, in_strides, shape_size, fn_id);
        cudaDeviceSynchronize();
    }
```

## **MiniTorch Automatic Differentiation**

In [38]:
from minitorch import SimpleBackend

MiniTorch tensors are connect operations in the computational graph as described in last lecture.

Note: this example uses SimpleBackend implemented in Python, which isn't nearly as good as your Cuda backend :)

Let's consider a simple example of $z = x * y$, the element-wise multiplication of two 1-d tensors.

Suppose this happened somewhere in your Neural Network.

During your backward pass, you've calculated your loss **$\frac{\partial loss}{\partial z}$.**

You want **$\frac{\partial loss}{\partial x}$** and **$\frac{\partial loss}{\partial y}$**

In [39]:
x = minitorch.tensor([1, 2, 3, 4, 5, 6], backend=SimpleBackend)

In [40]:
x.requires_grad_(True)

In [41]:
x


[1.000000 2.000000 3.000000 4.000000 5.000000 6.000000]

In [42]:
y = minitorch.tensor([2, 4, 6, 8, 10, 12], backend=SimpleBackend)

In [43]:
y.requires_grad_(True)

In [44]:
y


[2.000000 4.000000 6.000000 8.000000 10.000000 12.000000]

In [45]:
z = x * y

In [46]:
z


[2.000000 8.000000 18.000000 32.000000 50.000000 72.000000]

**How does MiniTorch calculate the gradients for each tensor?**

By using *reverse mode automatic differentiation* - let's examine how this is implemented in MiniTorch with the simple example above of

`z = x * y`

They contain important fields and methods for enabling automatic differentiation, including the
* `self.f` field - tells us what backend we're using
* `self.history` field - tracks the computation
* `self.grad` field - stores the gradient for the optimizer
* `chain_rule` method - lets us obtain partial adjoints for `backpropagate` to propagate from a node to its inputs



We are going to utilize the computation graph as a way to automatically compute derivatives of arbitrary python functions.

The trick behind this autodifferentiation is to implement the derivative of each individual function, and then utilize the chain rule to compute a derivative for any scale value.

**Let's examine the trace of the forward evaluation**
to get a better idea of how `z` got computed.

1. Open `minitorch/tensor.py:153` to find `__mul__`


```python
def __mul__(self, b: TensorLike) -> Tensor:
        return Mul.apply(self, self._ensure_tensor(b))
```



2. Open `minitorch/tensor_functions.py:99` to find
```python
class Mul(Function):
    @staticmethod
    def forward(ctx: Context, a: Tensor, b: Tensor) -> Tensor:
        ctx.save_for_backward(a, b)
        return a.f.mul_zip(a, b)

    @staticmethod
    def backward(ctx: Context, grad_output: Tensor) -> Tuple[Tensor, Tensor]:
        a, b = ctx.saved_values
        return (
            grad_output.f.mul_zip(b, grad_output),
            grad_output.f.mul_zip(a, grad_output),
        )
```


3. Open `minitorch/tensor_functions.py:33` to find

```python
class Function:
    @classmethod
    def _backward(cls, ctx: Context, grad_out: Tensor) -> Tuple[Tensor, ...]:
        return wrap_tuple(cls.backward(ctx, grad_out))  # type: ignore

    @classmethod
    def _forward(cls, ctx: Context, *inps: Tensor) -> Tensor:
        return cls.forward(ctx, *inps)  # type: ignore

    @classmethod
    def apply(cls, *vals: Tensor) -> Tensor:
        raw_vals = []
        need_grad = False
        for v in vals:
            if v.requires_grad():
                need_grad = True
            raw_vals.append(v.detach())

        # Create the context.
        ctx = Context(not need_grad)

        # Call forward with the variables.
        c = cls._forward(ctx, *raw_vals)
        
        back = None
        if need_grad:
            back = minitorch.History(cls, ctx, vals)
        return minitorch.Tensor(c._tensor, back, backend=c.backend)
```


So, we can see that Z is a new tensor with a **history**.

### **How do we get $\frac{\partial l}{\partial x}$ and $\frac{\partial l}{\partial y}$ given $\frac{\partial l}{\partial z}$?**


You've probably written this code in pytorch
```python
loss.sum().backward()
```

Since `loss.sum()` is still a tensor, you're calling the following method

```python
    def backward(self, grad_output: Optional[Tensor] = None) -> None:
        if grad_output is None:
            assert self.shape == (1,), "Must provide grad_output if non-scalar"
            grad_output = Tensor.make([1.0], (1,), backend=self.backend)
        backpropagate(self, grad_output)
```



You've reached a point in the graph you've constructed for the backward pass where you have the following gradient propagated back to $z$

In [47]:
dldz = minitorch.tensor([1, 1, 1, 1, 1, 1], backend=SimpleBackend)

Let's make use of `z.history`, which tells us:
1. The function used to calculate z
2. The context ie. intermediate values we need to calculate the gradients of z's inputs

In [48]:
z.history

History(last_fn=<class 'minitorch.tensor_functions.Mul'>, ctx=Context(no_grad=False, saved_values=(
[1.000000 2.000000 3.000000 4.000000 5.000000 6.000000], 
[2.000000 4.000000 6.000000 8.000000 10.000000 12.000000])), inputs=(
[1.000000 2.000000 3.000000 4.000000 5.000000 6.000000], 
[2.000000 4.000000 6.000000 8.000000 10.000000 12.000000]))

In your `backpropagate` function, you should use the `chain_rule` method of a tensor in `minitorch/tensor.py:373`.

```python
    def chain_rule(self, d_output: Any) -> Iterable[Tuple[Variable, Any]]:
        h = self.history
        assert h is not None
        assert h.last_fn is not None
        assert h.ctx is not None

        x = h.last_fn._backward(h.ctx, d_output)
        assert len(x) == len(h.inputs), f"Bug in function {h.last_fn}"
        return [
            (inp, inp.expand(self._ensure_tensor(d_in)))
            for inp, d_in in zip(h.inputs, x)
        ]
```


In [49]:
z.history.last_fn

minitorch.tensor_functions.Mul

In [50]:
# saved_values == (x, y)
z.history.ctx

Context(no_grad=False, saved_values=(
[1.000000 2.000000 3.000000 4.000000 5.000000 6.000000], 
[2.000000 4.000000 6.000000 8.000000 10.000000 12.000000]))

If $z = x*y$ then $\frac{\partial l}{\partial x} = \frac{\partial l}{\partial z} \frac{\partial z}{\partial x} = \frac{\partial l}{\partial z} y$


Open `minitorch/tensor_functions.py:99` to find
```python
class Mul(Function):
    @staticmethod
    def forward(ctx: Context, a: Tensor, b: Tensor) -> Tensor:
        ctx.save_for_backward(a, b)
        return a.f.mul_zip(a, b)

    @staticmethod
    def backward(ctx: Context, grad_output: Tensor) -> Tuple[Tensor, Tensor]:
        a, b = ctx.saved_values
        return (
            grad_output.f.mul_zip(b, grad_output),
            grad_output.f.mul_zip(a, grad_output),
        )
```

In [51]:
# This is exactly what's happening in backward!
dldx, dldy = z.history.last_fn._backward(z.history.ctx, dldz)

$\frac{\partial l}{\partial x} = \frac{\partial l}{\partial z} \frac{\partial z}{\partial x} = \frac{\partial l}{\partial z} y$
```python
dldx = tensor([1,1,1,1,1,1]) * tensor([2,4,6,8,10,12])
```

In [52]:
dldx # = dldz * y


[2.000000 4.000000 6.000000 8.000000 10.000000 12.000000]

In [53]:
(dldz * y) == dldx


[1.000000 1.000000 1.000000 1.000000 1.000000 1.000000]

In [70]:
dldy # = dldz * x


[1.000000 2.000000 3.000000 4.000000 5.000000 6.000000]

In [71]:
(dldz * x) == dldy


[1.000000 1.000000 1.000000 1.000000 1.000000 1.000000]

Defining functions in this way (Tensor in, Tensor out) is really powerful because it enables us to easily calculate the gradient of any python function that operates on Tensors eg. .sum(), .permute(), etc.

There are many things to be careful of when shapes change eg.
How do you calculate the gradient when you've done a `.sum(dim=2)`?

Backpropagation vs. Reverse Mode Automatic Differentiation
1. Modern deep learning frameworks don't use backpropagation on the forward computational graph.
